# 정부 공고문 rag 봇

In [1]:
# (Colab) RAG 파이프라인 + Hugging Face 오픈소스 모델 구동에 필요한 라이브러리 설치
# - transformers, accelerate, bitsandbytes: Hugging Face 모델을 효율적으로 로드하고 실행하기 위한 라이브러리
%pip install -q langchain pypdf faiss-cpu sentence-transformers transformers accelerate bitsandbytes -U langchain-community gdown langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7

## api 등 필요한 세팅

In [2]:
from google.colab import userdata
import os
try:
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("환경 변수가 성공적으로 설정되었습니다. ✅")
except Exception as e:
    print(f"🚨 환경 변수 설정 중 오류 발생: {e}")

환경 변수가 성공적으로 설정되었습니다. ✅


## (1/5) gdown으로 여러 PDF 로드 및 분할

In [3]:
# (1/5) gdown으로 여러 PDF 로드 및 분할
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# gdown 라이브러리 설치


# 다운로드할 파일 ID 리스트와 파일명 리스트 (샘플 10개 테스트용)
# https://drive.google.com/file/d/1FS0JkuIy8mIQY2HrMODloFIYq-zdU7VI/view?usp=sharing
# https://drive.google.com/file/d/1UpOZaVY5Oqbxlx3aPjiBflH3ElWaFgQv/view?usp=sharing
# https://drive.google.com/file/d/1HijTvT_bkkEvV8nN5RICThp248haf9Kj/view?usp=sharing
# https://drive.google.com/file/d/1Ksa6s-luSI30-C54uJq6byHTCcRpbKB9/view?usp=sharing
# https://drive.google.com/file/d/1xZIAsvZHgwvg2dmHCQCCIegAGGm2k7e1/view?usp=sharing
# https://drive.google.com/file/d/1K45deO3xTcMPKJFOp4LKpGijVvr-Maww/view?usp=sharing
# https://drive.google.com/file/d/12RRJLZ6-2mCEvqsfdHPIb4sVnj0GlrSZ/view?usp=sharing
# https://drive.google.com/file/d/1CnVXGIpSpjRwQhfMwuG3uPMQ8LEuHmNG/view?usp=sharing
# https://drive.google.com/file/d/1ZGXSpFbJDp1UgVmwklw_SvYQtMFcDD1-/view?usp=sharing
# https://drive.google.com/file/d/1Z_3yxybPVa-fRHkoOITctmspMpOZF6Xi/view?usp=sharing


file_infos = [
    {'id': '1FS0JkuIy8mIQY2HrMODloFIYq-zdU7VI', 'name': "(iris사전공시용)+(기획보고서)+로봇친화형+건축물+설계+및+리모델링+핵심기술+개발+기획(R&D).pdf"},
    {'id': '1UpOZaVY5Oqbxlx3aPjiBflH3ElWaFgQv', 'name': "(공고문)디지털 정부혁신을 위한 서비스 아이디어 공모.pdf"},
    {'id': '1HijTvT_bkkEvV8nN5RICThp248haf9Kj', 'name': "(기획보고서)AI-XR+기반+비행장+통합관제+운용기술+개발(R&D)_공고용.pdf"},
    {'id': '1Ksa6s-luSI30-C54uJq6byHTCcRpbKB9', 'name': "2023년 노사문화유공 등 정부포상 수상자 공고문.pdf"},
    {'id': '1xZIAsvZHgwvg2dmHCQCCIegAGGm2k7e1', 'name': "2025년_K-City-Network_해외실증형_사업_공고문(국문).pdf"},
    {'id': '1K45deO3xTcMPKJFOp4LKpGijVvr-Maww', 'name': "2025년도_중앙부처_및_지자체_창업지원사업_통합공고문(제2024-626호,_2024.12.31.).pdf"},
    {'id': '12RRJLZ6-2mCEvqsfdHPIb4sVnj0GlrSZ', 'name': "교통신기술_지정_신청기술_접수결과_및_공고_(안)(교통-137).pdf"},
    {'id': '1CnVXGIpSpjRwQhfMwuG3uPMQ8LEuHmNG', 'name': "국토교통부공고제2025-841호(건설신기술+지정+신청(2728)).pdf"},
    {'id': '1ZGXSpFbJDp1UgVmwklw_SvYQtMFcDD1-', 'name': "붙임 . 2025년 제3차 수출용 출판 홍보자료샘플 지원 사업 공고문.pdf"},
    {'id': '1Z_3yxybPVa-fRHkoOITctmspMpOZF6Xi', 'name': "★+[국토교통과학기술진흥원]+계약업무+제도개선+사항+안내(25년+1월+기준).pdf"},

]

# gdown으로 파일 다운로드
for info in file_infos:
    !gdown --id {info['id']} -O "{info['name']}"

# 모든 PDF 문서를 담을 리스트 초기화
all_documents = []

# 각 PDF 파일을 순서대로 로드하여 리스트에 추가
for info in file_infos:
    if os.path.exists(info['name']):
        print(f"'{info['name']}' 로드 중...")
        loader = PyPDFLoader(info['name'])
        all_documents.extend(loader.load())
    else:
        print(f"경고: '{info['name']}' 파일이 다운로드되지 않았습니다.")

# 텍스트 분할기 생성
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 합쳐진 모든 문서를 분할
split_docs = text_splitter.split_documents(all_documents)

# 분할 결과 확인
print(f"\n총 로드된 PDF 페이지 수: {len(all_documents)}")
print(f"분할 후 총 텍스트 청크 수: {len(split_docs)}")
print("\n--- 첫 번째 청크 샘플 ---")
if split_docs:
    print(split_docs[0].page_content)

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1FS0JkuIy8mIQY2HrMODloFIYq-zdU7VI
To: /content/(iris사전공시용)+(기획보고서)+로봇친화형+건축물+설계+및+리모델링+핵심기술+개발+기획(R&D).pdf
100% 397k/397k [00:00<00:00, 123MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1UpOZaVY5Oqbxlx3aPjiBflH3ElWaFgQv
To: /content/(공고문)디지털 정부혁신을 위한 서비스 아이디어 공모.pdf
100% 11.5M/11.5M [00:00<00:00, 160MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to

ERROR:pypdf._cmap:Advanced encoding /KSCms-UHC-H not implemented yet
ERROR:pypdf._cmap:Advanced encoding /KSCms-UHC-H not implemented yet
ERROR:pypdf._cmap:Advanced encoding /KSCms-UHC-H not implemented yet
ERROR:pypdf._cmap:Advanced encoding /KSCms-UHC-H not implemented yet


'2023년 노사문화유공 등 정부포상 수상자 공고문.pdf' 로드 중...
'2025년_K-City-Network_해외실증형_사업_공고문(국문).pdf' 로드 중...
'2025년도_중앙부처_및_지자체_창업지원사업_통합공고문(제2024-626호,_2024.12.31.).pdf' 로드 중...
'교통신기술_지정_신청기술_접수결과_및_공고_(안)(교통-137).pdf' 로드 중...
'국토교통부공고제2025-841호(건설신기술+지정+신청(2728)).pdf' 로드 중...
'붙임 . 2025년 제3차 수출용 출판 홍보자료샘플 지원 사업 공고문.pdf' 로드 중...
'★+[국토교통과학기술진흥원]+계약업무+제도개선+사항+안내(25년+1월+기준).pdf' 로드 중...

총 로드된 PDF 페이지 수: 625
분할 후 총 텍스트 청크 수: 996

--- 첫 번째 청크 샘플 ---
디지털 정부혁신을 위한 서비스 아이디어 공모행정안전부와한국정보화진흥원은국민이희망하는새로운디지털정부아이디어를공공서비스에적용하고자「디지털정부혁신을위한서비스아이디어」를다음과같이공모하오니관심있는국민여러분의많은참여바랍니다. 2020.3.30.행정안전부장관1.1. 추진 목적 추진 목적□국민에게희망하는정부서비스를제안토록함으로써정부혁신에대한국민적관심을유도하고새로운디지털정부서비스발굴도모2.2. 공모계획 공모계획□공모개요○공모주제:디지털정부혁신을위한서비스아이디어*디지털정부참고:행정안전부누리집(mois.go.kr)전자정부전시체험관-전자정부50주년(egovhall.kr/kor)○공모일정:2020.3.30.(월)~5.7.(목)○공모부문 공모 부문공모 내용참가자격일반국민 부문디지털 정부혁신을 위한 정부 서비스대한민국 국민 누구나민간기업 부문민간주도형 디지털 정부 서비스디 지 털  정 부  서 비 스 에  관 심  있 는  민 간 기 업*일반국민부문:부처,지자체,공공기관,일반국민등참여에대한제한이없음*민간기업부문:대기업,중소기업,스타트업(Start-up)등참여에제한없음
행정안전부공고제2020-208호


## (2/5) 임베딩 및 벡터 DB 구축 (Hugging Face 모델 사용)

In [4]:
# (2/5) 임베딩 및 벡터 DB 구축 (Hugging Face 모델 사용)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# (1) 임베딩 모델 선택: 문장을 의미가 담긴 벡터로 변환하는 모델
# 'ko-sbert-nli': 한국어 문장 임베딩에 성능이 좋다고 알려진 모델 중 하나
# device='cuda' 옵션으로 GPU를 사용하도록 명시하여 속도 향상
model_name = "jhgan/ko-sbert-nli"
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

# (2) 벡터 데이터베이스 생성 및 저장
# FAISS.from_documents: 분할된 문서(split_docs)들을 임베딩 모델로 벡터화하고,
# FAISS 벡터 DB에 한 번에 저장해주는 편리한 함수
print("텍스트 청크들을 벡터로 변환하여 DB에 저장)")
vector_db = FAISS.from_documents(split_docs, embedding_model)

print("벡터 데이터베이스 생성 완료!")

/tmp/ipython-input-4-3135213355.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

텍스트 청크들을 벡터로 변환하여 DB에 저장)
벡터 데이터베이스 생성 완료!


## (3/5) 오픈소스 LLM을 사용한 RAG 파이프라인 생성

In [5]:
# (3/5) OpenAI LLM을 사용한 RAG 파이프라인 생성
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# (1) Retriever와 (2) Prompt는 이전과 동일
retriever = vector_db.as_retriever()
template = """
[역할]
당신은 대한민국 정부 및 공공기관의 정책 공고문을 분석하고 핵심 정보를 추출하는 데 매우 능숙한 '정책 분석 전문 AI'입니다. 당신의 임무는 사용자가 궁금해하는 내용을 제공된 문서 안에서 찾아, 가장 정확하고 이해하기 쉽게 설명하는 것입니다.

[작업 절차]
1. 사용자의 '질문' 의도를 명확하게 파악합니다.
2. 주어진 '공고문 내용' 전체를 꼼꼼하게 읽고, 질문과 가장 관련이 깊은 핵심 정보들을 모두 찾습니다.
3. 찾아낸 정보들을 바탕으로, 아래 '출력 형식'에 맞춰 답변을 생성합니다.
4. 만약 '공고문 내용'에서 질문에 대한 답을 찾을 수 없다면, 절대 추측해서 답변하지 말고 "제공된 문서에서는 해당 내용을 찾을 수 없습니다."라고 솔직하게 답변해야 합니다.

[공고문 내용]
{context}

[질문]
{question}

[출력 형식]
**1. 핵심 요약:**
(질문에 대한 가장 핵심적인 내용을 1~2문장으로 요약)

**2. 세부 내용:**
(핵심 요약에 대한 구체적인 내용을 항목별로 나누어 설명)
- **(항목 1):** (내용)
- **(항목 2):** (내용)
- **(항목 3):** (내용)
"""
prompt = ChatPromptTemplate.from_template(template)

# (3) LLM (언어 모델) 선택: gpt-4.1-nano
# temperature=0으로 설정하여, 사실 기반의 일관된 답변을 생성하도록 유도
llm = ChatOpenAI(model_name="gpt-4.1-nano", temperature=0)

# (4) RAG Chain (RAG 체인) 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("OpenAI LLM을 사용한 RAG 체인 생성 완료! 이제 질문할 수 있습니다.")

OpenAI LLM을 사용한 RAG 체인 생성 완료! 이제 질문할 수 있습니다.


## (4/5) RAG 체인에 질문하고 답변 받기

In [6]:
# (4/5) RAG 체인에 질문하고 답변 받기

# 질문 내용: PDF에 있을 법한 내용으로 질문을 만들어보자
question = "디지털 정부혁신을 위한 아이디어 공모의 목적은 무엇인가요?"

# rag_chain.invoke(): 질문을 입력하여 RAG 파이프라인 전체를 실행하고 최종 답변을 받음
print(f"질문: {question}\n")
print("답변 생성 중... 잠시만 기다려주세요.")
print("-" * 30)

response = rag_chain.invoke(question)

# 최종 답변 출력
print(response)

질문: 디지털 정부혁신을 위한 아이디어 공모의 목적은 무엇인가요?

답변 생성 중... 잠시만 기다려주세요.
------------------------------
**1. 핵심 요약:**  
디지털 정부혁신을 위한 아이디어 공모의 목적은 국민과 민간기업이 디지털 기술을 활용하여 정부 서비스의 혁신과 개선 방안을 제안함으로써 정부혁신과 국민 편익 증진을 유도하는 데 있습니다.

**2. 세부 내용:**  
- **국민 참여 유도:** 국민이 희망하는 정부 서비스와 업무 방식을 디지털 우선 관점(Digital First)으로 제안하여 정부혁신에 대한 국민의 관심과 참여를 촉진합니다.  
- **신기술 활용 및 서비스 개선:** 인공지능, 빅데이터 등 첨단 신기술을 활용하여 정부 서비스를 개선하거나 새롭게 제안하는 아이디어를 발굴합니다.  
- **민간기업과의 협력 촉진:** 민간기업이 혁신적인 아이디어로 디지털 정부 서비스를 기획·구축하고, 정부는 이를 활용하는 새로운 패러다임을 확산하여 공공서비스의 질적 향상과 산업 발전을 도모합니다.


## (5/5) 향상된 기능 구현 및 전체 요약

In [7]:
# (5/5) 특정 공고문 전체 요약 요청 (스트리밍 방식)

# 질문 내용: 특정 파일의 전체 요약을 요청하는 질문으로 변경
question = "'붙임 . 2025년 제3차 수출용 출판 홍보자료샘플 지원 사업 공고문'의 전체 내용을 핵심만 요약해줘."

print(f"질문: {question}\n")
print("답변 생성 중... (스트리밍 시작)")
print("-" * 30)

# .stream() 메소드를 사용하여 스트리밍으로 답변을 실시간 출력
for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)

print("\n" + "-" * 30)
print("스트리밍 종료!")

질문: '붙임 . 2025년 제3차 수출용 출판 홍보자료샘플 지원 사업 공고문'의 전체 내용을 핵심만 요약해줘.

답변 생성 중... (스트리밍 시작)
------------------------------
**1. 핵심 요약:**  
한국출판문화산업진흥원은 2025년 제3차 수출용 출판 홍보자료(샘플) 지원 사업을 통해 국내 출판사의 해외 시장접근성 강화를 위해 번역비를 지원하고, 최대 15건의 샘플 제작을 지원합니다.

**2. 세부 내용:**  
- **지원 대상:** 해외 진출을 희망하는 국내 출판사 및 에이전시(임프린트 등록 포함)로, 사업자등록번호가 동일한 경우 한 출판사로 간주됩니다.  
- **지원 규모:** 샘플별 최대 200만 원, 연간 최대 15건 신청 가능하며, 번역실비(감수비 포함)를 지원합니다.  
- **지원 내용:** 샘플 서지정보, 작가·작품 소개, 마케팅 정보, 도서 일부 발췌 번역 및 감수비 지원, 온라인 신청 양식 제공, 최대 20,000자 번역 지원, 일부 도서(아동도서·그림책 등)는 전권 번역 가능하며, 외국어는 영어, 중국어, 일본어, 프랑스어, 베트남어, 러시아어, 독일어 중 선택.  
- **기타 조건:** 진흥원 출판유통통합전산망 미등록 도서 접수 불가, 미출간 원고 제외, 시리즈는 1종으로 신청, 번역가 프로필 사전 제출, 감수비는 번역비의 20%임.
------------------------------
스트리밍 종료!
